In [1]:
import os
train_path = '../../raw_train_artifact'
test_path = '../../raw_test_artifact'
embedding_path = '../../embedding_artifact'
input_path = '../../input_artifact'
input_split_path = '../../input_artifact/input_split'
model_path = '../../model_artifact'
output_path = '../../output_artifact'

In [2]:
import sys
import gc
gc.enable()
import time
import re

import numpy as np
import pandas as pd
pd.set_option('display.max_columns',120)
pd.set_option('display.max_rows',2000)
pd.set_option('precision',5)
pd.set_option('float_format', '{:.5f}'.format)

import tqdm
import joblib
import json

from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, roc_auc_score
from gensim.models import Word2Vec
import torch
from torch import nn
import torch.nn.functional as F

In [3]:
import logging

log_path = '[1.1]Hugging Face Transformer Experiment.log'
    
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

formatter = logging.Formatter('%(asctime)s %(levelname)-s: %(message)s', datefmt='%H:%M:%S')

fh = logging.FileHandler(log_path)
fh.setLevel(logging.INFO)
fh.setFormatter(formatter)
logger.addHandler(fh)

sh = logging.StreamHandler(sys.stdout)
sh.setLevel(logging.INFO)
sh.setFormatter(formatter)
logger.addHandler(sh)

logger.info(f'Restart notebook\n==========================\n{time.ctime()}\n==========================')

16:27:39 INFO: Restart notebook
Sun Jun  7 16:27:39 2020


In [4]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info('Device in Use: {}'.format(DEVICE))
torch.cuda.empty_cache()
t = torch.cuda.get_device_properties(DEVICE).total_memory/1024**3
c = torch.cuda.memory_cached(DEVICE)/1024**3
a = torch.cuda.memory_allocated(DEVICE)/1024**3
logger.info('CUDA Memory: Total {:.2f} GB, Cached {:.2f} GB, Allocated {:.2f} GB'.format(t,c,a))

16:27:39 INFO: Device in Use: cuda
16:27:39 INFO: CUDA Memory: Total 8.00 GB, Cached 0.00 GB, Allocated 0.00 GB


In [5]:
def get_torch_module_num_of_parameter(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    return params

In [6]:
from transformers.modeling_bert import BertConfig, BertEncoder, BertAttention,BertSelfAttention,BertLayer,BertPooler

## Transformer Classifier

In [248]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn.init import xavier_uniform_
from torch.nn import TransformerEncoder, TransformerEncoderLayer

class Positional_Encoding_Layer(nn.Module):
	"""
	Positional encoding using sine and cosine as described in "Attention is all you need".
	"""
	def __init__(self, d_model, max_seq_len=512, dropout=0.1):
		"""
		Formula:
		| PE(pos,2i) = sin(pos/10000**(2*i/d_model))
		| PE(pos,2i+1) = cos(pos/10000**(2*i/d_model))
		"""
		super(Positional_Encoding_Layer, self).__init__()
		self.d_model = d_model
		self.dropout = dropout
		self.max_seq_len = max_seq_len

		self.dropout_layer = nn.Dropout(p=dropout)
		pe = torch.zeros(max_seq_len, d_model)                                                       # (max_seq_len, d_model)
		position = torch.arange(0, max_seq_len, dtype=torch.float).unsqueeze(1)                      # (max_seq_len, 1)
		div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))     # (d_model/2)
		pe[:, 0::2] = torch.sin(position * div_term)
		pe[:, 1::2] = torch.cos(position * div_term) 
		pe = pe.unsqueeze(0).transpose(0, 1)                                                         # (max_seq_len, 1, d_model)
		self.register_buffer('pe', pe)

	def forward(self, inp):
		inp = inp + self.pe[:inp.size(0), :]                                                         # (n_step, batch_size, d_model)
		return self.dropout_layer(inp)

class Transformer_Encoder_Extraction_Layer(nn.Module):
	"""
	Transformer encoder as described in "Attention is all you need".
	"""
	def __init__(self, n_enc_layer, embed_size, n_head, intermediate_size, max_seq_len=512, dropout=0.1, **kwargs):
		super(Transformer_Encoder_Extraction_Layer, self).__init__(**kwargs)
		assert embed_size%n_head==0

		self.n_enc_layer = n_enc_layer
		self.embed_size = embed_size
		self.max_seq_len = max_seq_len
		self.n_head = n_head
		self.intermediate_size = intermediate_size
		self.dropout = dropout

		self.positional_encoder = Positional_Encoding_Layer(embed_size, max_seq_len=max_seq_len)
		transformer_encoder_layer = TransformerEncoderLayer(embed_size, n_head, dim_feedforward=intermediate_size, dropout=dropout)
		self.transformer_encoder = TransformerEncoder(transformer_encoder_layer, n_enc_layer)

		self._init_weights()

	def _init_weights(self):
		for p in self.parameters():
			if p.dim() > 1:
				xavier_uniform_(p)

	def forward(self, inp, inp_padding_mask=None):
		inp = inp * np.sqrt(self.embed_size)                                           # (batch_size, n_step, embed_size)
		inp = self.positional_encoder(inp.permute(1, 0, 2))                            # (n_step, batch_size, embed_size)
		out = self.transformer_encoder(inp, src_key_padding_mask=inp_padding_mask)     # (n_step, batch_size, embed_size)
		return out.permute(1, 0, 2)

class MLP_Classification_Layer(nn.Module):
	"""
	Multilayer Perception Classification Layer
	- Layer 1: Linear + Batchnorm + ReLU + Dropout
	- Layer 2: Linear + Batchnorm + ReLU + Dropout
	- Layer 3: Linear
	"""
	def __init__(self, inp_size, out_size, dropout=0.4, **kwargs):
		super(MLP_Classification_Layer, self).__init__(**kwargs)
		self.inp_size = inp_size
		self.out_size = out_size
		self.dropout = dropout
		
		self.mlp_1 = nn.Linear(inp_size, 1024)
		self.batchnorm_1 = nn.BatchNorm1d(1024)
		self.mlp_dropout_1 = nn.Dropout(p=dropout)
		self.mlp_2 = nn.Linear(1024, 512)
		self.batchnorm_2 = nn.BatchNorm1d(512)
		self.mlp_dropout_2 = nn.Dropout(p=dropout)
		self.mlp_3 = nn.Linear(512, out_size)
	
	def _init_weights(self):
		initrange = 0.1
		self.mlp_1.weight.data.uniform_(-initrange, initrange)
		self.mlp_1.bias.data.zero_()
		self.mlp_2.weight.data.uniform_(-initrange, initrange)
		self.mlp_2.bias.data.zero_()
		self.mlp_3.weight.data.uniform_(-initrange, initrange)
		self.mlp_3.bias.data.zero_()

	def forward(self, inp):
		mlp_out = self.mlp_1(inp)                                                         # (batch_size, 1024)
		mlp_out = self.mlp_dropout_1(F.relu(self.batchnorm_1(mlp_out)))                   # (batch_size, 1024)
		mlp_out = self.mlp_2(mlp_out)                                                     # (batch_size, 512)
		mlp_out = self.mlp_dropout_2(F.relu(self.batchnorm_2(mlp_out)))                   # (batch_size, 512)
		mlp_out = self.mlp_3(mlp_out)                                                     # (batch_size, out_size)
		return mlp_out   
	
class Transformer_Encoder_Classifier(nn.Module):
	"""
	Transformer Encoder + Multilayer Perception for Classification
	"""
	def __init__(self, embed_size, out_size, n_enc_layer, n_head, intermediate_size, device, transformer_dropout=0.1, mlp_dropout=0.4, **kwargs):
		super(Transformer_Encoder_Classifier, self).__init__(**kwargs)
		
		self.embed_size = embed_size
		self.out_size = out_size
		self.n_enc_layer = n_enc_layer
		self.n_head = n_head
		self.intermediate_size = intermediate_size
		self.device = device
		self.transformer_dropout = transformer_dropout
		self.mlp_dropout = mlp_dropout

		self.encoder_layer = Transformer_Encoder_Extraction_Layer(n_enc_layer, embed_size, n_head, intermediate_size, dropout=transformer_dropout)
		self.classification_layer = MLP_Classification_Layer(embed_size, out_size, dropout=mlp_dropout)

	def get_padding_mask(self, batch_size, seq_len, inp_last_idx):
		padding_mask = np.ones((batch_size, seq_len))
		for index, last_idx in enumerate(inp_last_idx):
			padding_mask[index,:last_idx+1] = 0
		return torch.from_numpy(padding_mask).bool().to(self.device)

	def forward(self, inp_embed, inp_last_idx):
		assert inp_embed.shape[0] == inp_last_idx.shape[0]
		batch_size = inp_embed.shape[0]
		seq_len = inp_embed.shape[1]
		inp_padding_mask = self.get_padding_mask(batch_size, seq_len, inp_last_idx)
		out = self.encoder_layer(inp_embed, inp_padding_mask=inp_padding_mask)               # (batch_size, n_step, embed_size)
		pooled_buf = []
		for index, last_idx in enumerate(inp_last_idx):
			pooled_buf.append(torch.mean(out[index,:last_idx+1,:], dim=0))
		out = torch.stack(pooled_buf)                                                        # (batch_size, embed_size)
		out = self.classification_layer(out)                                                 # (batch_size, out_size)
		return out

In [249]:
model = Transformer_Encoder_Classifier(256, 2, 6, 8, 2048, torch.device('cpu'))
get

In [250]:
inp = torch.from_numpy(np.concatenate([p, np.zeros((100, 4, 256))], axis=1)).float()
inp_last_idx = np.array([5 for _ in range(100)])

In [252]:
model(inp, inp_last_idx)

tensor([[-7.6068e-01, -4.1554e-02],
        [ 1.0114e-01,  2.2231e-01],
        [-9.9992e-02, -1.4431e-01],
        [ 4.9414e-01,  6.9754e-03],
        [-8.7818e-01, -7.5826e-01],
        [-4.2960e-01, -3.7006e-01],
        [ 3.0174e-02,  1.6330e-01],
        [-1.2335e-01,  2.0211e-01],
        [ 5.0249e-02,  1.0301e-01],
        [-9.6979e-02,  9.5880e-02],
        [-2.7329e-01, -1.8086e-01],
        [-1.0915e+00, -1.7455e-01],
        [ 2.5860e-01, -2.1152e-01],
        [-1.2127e-01, -2.3629e-01],
        [-2.8017e-01, -2.8132e-01],
        [-2.5161e-01, -6.1382e-01],
        [-1.2896e-01, -3.4738e-04],
        [ 1.0753e+00, -7.7634e-02],
        [-5.3120e-01, -4.6385e-02],
        [ 3.2261e-01, -3.1052e-01],
        [-3.0064e-01,  3.7959e-01],
        [-9.7347e-01, -4.9769e-01],
        [-3.1129e-01, -4.8083e-01],
        [-9.3990e-01,  6.4359e-01],
        [ 7.3314e-01, -2.5928e-01],
        [-5.4152e-02, -1.1831e-01],
        [-2.7646e-01,  3.3617e-01],
        [ 2.6645e-02, -4.052

In [ ]:
torch.ones(1,10).shape

In [218]:
torch.ones(1,10).squeeze(0).shape

torch.Size([10])

In [211]:
torch.stack([torch.mean(torch.ones(1,10,20), dim=1).squeeze(0), torch.mean(torch.ones(1,10,20), dim=1).squeeze(0)]).shape

torch.Size([2, 20])

## Transformer

In [8]:
torch.arange(0, 10, dtype=torch.float).unsqueeze(1).shape

torch.Size([10, 1])

In [14]:
d_model = 13
torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model)).shape

torch.Size([7])

In [179]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn.init import xavier_uniform_
from torch.nn import TransformerEncoder, TransformerEncoderLayer

class Positional_Encoding_Layer(nn.Module):
	"""
	Positional encoding using sine and cosine as described in "Attention is all you need".
	"""
	def __init__(self, d_model, max_seq_len=128, dropout=0.1):
		"""
		Formula:
		| PE(pos,2i) = sin(pos/10000**(2*i/d_model))
		| PE(pos,2i+1) = cos(pos/10000**(2*i/d_model))
		"""
		super(Positional_Encoding_Layer, self).__init__()
		self.d_model = d_model
		self.dropout = dropout
		self.max_seq_len = max_seq_len

		self.dropout_layer = nn.Dropout(p=dropout)
		pe = torch.zeros(max_seq_len, d_model)                                                       # (max_seq_len, d_model)
		position = torch.arange(0, max_seq_len, dtype=torch.float).unsqueeze(1)                      # (max_seq_len, 1)
		div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))     # (d_model/2)
		pe[:, 0::2] = torch.sin(position * div_term)
		pe[:, 1::2] = torch.cos(position * div_term) 
		pe = pe.unsqueeze(0).transpose(0, 1)                                                         # (max_seq_len, 1, d_model)
		self.register_buffer('pe', pe)

	def forward(self, inp):
		inp = inp + self.pe[:inp.size(0), :]                                                         # (n_step, batch_size, d_model)
		return self.dropout_layer(inp)

class Transformer_Encoder_Extraction_Layer(nn.Module):
	"""
	Transformer encoder as described in "Attention is all you need".
	"""
	def __init__(self, n_enc_layer, embed_size, n_head, intermediate_size, max_seq_len=512, dropout=0.1, **kwargs):
		super(Transformer_Encoder_Extraction_Layer, self).__init__(**kwargs)
		assert embed_size%n_head==0

		self.n_enc_layer = n_enc_layer
		self.embed_size = embed_size
		self.max_seq_len = max_seq_len
		self.n_head = n_head
		self.intermediate_size = intermediate_size
		self.dropout = dropout

		self.positional_encoder = Positional_Encoding_Layer(embed_size, max_seq_len=max_seq_len)
		transformer_encoder_layer = TransformerEncoderLayer(embed_size, n_head, dim_feedforward=intermediate_size, dropout=dropout)
		self.transformer_encoder = TransformerEncoder(transformer_encoder_layer, n_enc_layer)

		self._init_weights()

	def _init_weights(self):
		for p in self.parameters():
			if p.dim() > 1:
				xavier_uniform_(p)

	def forward(self, inp, inp_padding_mask=None):
		inp = inp * np.sqrt(self.embed_size)                                           # (batch_size, n_step, embed_size)
		inp = self.positional_encoder(inp.permute(1, 0, 2))                            # (n_step, batch_size, embed_size)
		out = self.transformer_encoder(inp, src_key_padding_mask=inp_padding_mask)     # (n_step, batch_size, embed_size)
		return out.permute(1, 0, 2)

In [180]:
def get_torch_module_num_of_parameter(model):
	"""
	Get # of parameters in a torch module.
	"""
	model_parameters = filter(lambda p: p.requires_grad, model.parameters())
	params = sum([np.prod(p.size()) for p in model_parameters])
	return params

In [183]:
model = Transformer_Encoder_Extraction_Layer(6, 256, 8, 2048)
get_torch_module_num_of_parameter(model)

7890432

In [184]:
p = np.random.standard_normal((100,6,256))
inp_1 = torch.from_numpy(np.concatenate([p, np.zeros((100, 4, 256))], axis=1)).float()
inp_2 = torch.from_numpy(np.concatenate([p, np.ones((100, 4, 256))], axis=1)).float()
pad_mask = torch.from_numpy(np.concatenate([np.zeros((100, 6)), np.ones((100,4))], axis=1)).bool()

In [185]:
def get_padding_mask(batch_size, seq_len, inp_last_idx):
    padding_mask = np.ones((batch_size, seq_len))
    for index, last_idx in enumerate(inp_last_idx):
        padding_mask[index,:last_idx+1] = 0
    return torch.from_numpy(padding_mask).bool()

In [186]:
get_padding_mask(3,10,[5,6,7])

tensor([[False, False, False, False, False, False,  True,  True,  True,  True],
        [False, False, False, False, False, False, False,  True,  True,  True],
        [False, False, False, False, False, False, False, False,  True,  True]])

In [187]:
a

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [188]:
model.eval()
out_1 = model(inp_1, inp_padding_mask=pad_mask)
out_2 = model(inp_2, inp_padding_mask=pad_mask)

## BERT

In [150]:
from transformers import BertModel, BertConfig

In [151]:
configuration = BertConfig(vocab_size=1, hidden_size=4, num_hidden_layers=8, num_attention_heads=2, intermediate_size=2048)
model = BertModel(configuration)

In [152]:
inp = torch.from_numpy(np.concatenate([np.random.standard_normal((1, 5, 4)), np.zeros((1, 5, 4))], axis=1)).float()
mask = torch.from_numpy(np.concatenate([np.zeros((1, 5)), np.zeros((1, 5))], axis=1)).float()

In [153]:
inp

tensor([[[-0.4829,  0.3867,  1.2085,  1.0879],
         [-0.4869, -2.1700,  0.7577, -0.3682],
         [-1.4087, -0.4176, -0.6754,  0.0763],
         [-0.1583, -0.2029,  0.6745, -0.6823],
         [ 0.4552,  0.5747, -0.3314, -1.9143],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]])

In [154]:
out= model(inputs_embeds=inp, attention_mask=mask)

In [155]:
inp.shape

torch.Size([1, 10, 4])

In [156]:
out[0]

tensor([[[-1.5516, -0.2037,  0.9508,  0.8045],
         [ 0.1553, -1.5379,  1.2636,  0.1190],
         [-0.6950,  0.0802, -0.9813,  1.5961],
         [-0.0377, -0.2082,  1.5238, -1.2779],
         [ 0.7606,  0.8213,  0.0738, -1.6557],
         [ 0.2063,  0.3378,  1.0872, -1.6313],
         [ 0.4727,  0.4431,  0.7993, -1.7151],
         [ 0.6379,  0.8671,  0.1718, -1.6768],
         [ 0.5959,  0.3670,  0.7528, -1.7157],
         [ 1.2472,  0.6829, -0.7249, -1.2052]]],
       grad_fn=<NativeLayerNormBackward>)

## Experiment with torch padding

In [62]:
from transformers import AlbertModel, AlbertConfig

In [63]:
configuration = AlbertConfig()
model = AlbertModel(configuration)

In [64]:
get_torch_module_num_of_parameter(model)

222595584

In [41]:
inp = torch.from_numpy(np.concatenate([np.ones((10, 5, 128)), np.zeros((10, 5, 128))], axis=1)).float()

In [44]:
out= model(inputs_embeds=inp)

In [45]:
out[0].shape

torch.Size([10, 10, 4096])

In [48]:
out[0][1].detach().numpy()

array([[-1.0415586 , -0.700448  , -0.23963146, ..., -0.34111258,
        -0.88970816,  2.0453863 ],
       [-1.0419453 , -0.7001726 , -0.24012905, ..., -0.34073353,
        -0.8900057 ,  2.0456576 ],
       [-1.0417546 , -0.7000878 , -0.24050954, ..., -0.34136513,
        -0.8902673 ,  2.0448096 ],
       ...,
       [-1.0415502 , -0.70019853, -0.23981182, ..., -0.34107703,
        -0.8901159 ,  2.0457187 ],
       [-1.0413502 , -0.6995464 , -0.240383  , ..., -0.34042382,
        -0.8905616 ,  2.0456984 ],
       [-1.0413843 , -0.70032   , -0.23944165, ..., -0.34181798,
        -0.89004594,  2.0447192 ]], dtype=float32)